# Imports, load data

In [1]:
import time
import warnings
from collections import namedtuple
import os
import pickle as pkl
import ast

from IPython.display import display, clear_output
from ipywidgets import IntProgress

import numpy as np
import matplotlib.pyplot as plt

from src.rbm import RBM
from src.utils import load_model

In [2]:
%load_ext autoreload
%autoreload 2

In [15]:
rbm_AFR = load_model()

[0] {'ni': '3', 'bs': '10', 'note': 'td50', 'nc': '50', 'lr': '0.05', 'pop': 'EAS.p'}
[1] {'ni': '3', 'bs': '10', 'note': 'td50', 'nc': '50', 'lr': '0.05', 'pop': 'EUR.p'}
[2] {'ni': '3', 'bs': '10', 'note': 'td50', 'nc': '50', 'lr': '0.05', 'pop': 'AFR.p'}
[3] {'ni': '3', 'bs': '10', 'note': 'td50', 'nc': '50', 'lr': '0.05', 'pop': 'AMR.p'}
Which file number?1


In [16]:
H_AFR = np.loadtxt('data/H_AFR.txt')
H_AMR = np.loadtxt('data/H_AMR.txt')
H_EAS = np.loadtxt('data/H_EAS.txt')
H_EUR = np.loadtxt('data/H_EUR.txt')

# Train

In [4]:
p2H = {'AFR': H_AFR, 'AMR': H_AMR, 'EAS': H_EAS, 'EUR': H_EUR}

params = [
    {'n_components': 50, 'n_iter': 3, 'learning_rate': 0.05, 'population': p, 
     'verbose': True, 'note': 'td50'} for p in ['AFR', 'AMR', 'EAS', 'EUR']
]

for p_dict in params:
    rbm = RBM(**p_dict)
    H = p2H[p_dict['population']][:-50]
    rbm.fit(p2H[p_dict['population']])
    rbm.save()

[RBM] Iteration 1, pseudo-likelihood = -2510.76, time = 1.29s
[RBM] Iteration 2, pseudo-likelihood = -4752.79, time = 1.38s
[RBM] Iteration 3, pseudo-likelihood = -5547.47, time = 1.47s
Saved RBM(batch_size=10, learning_rate=0.05, n_components=50, n_iter=3, note='td50',
  population='AFR', random_state=None, verbose=True)
[RBM] Iteration 1, pseudo-likelihood = -9718.40, time = 0.95s
[RBM] Iteration 2, pseudo-likelihood = -6023.31, time = 1.09s
[RBM] Iteration 3, pseudo-likelihood = -4964.54, time = 1.22s
Saved RBM(batch_size=10, learning_rate=0.05, n_components=50, n_iter=3, note='td50',
  population='AMR', random_state=None, verbose=True)
[RBM] Iteration 1, pseudo-likelihood = -9511.87, time = 1.49s
[RBM] Iteration 2, pseudo-likelihood = -4505.47, time = 1.60s
[RBM] Iteration 3, pseudo-likelihood = -3324.37, time = 1.61s
Saved RBM(batch_size=10, learning_rate=0.05, n_components=50, n_iter=3, note='td50',
  population='EAS', random_state=None, verbose=True)
[RBM] Iteration 1, pseudo-li

# Impute

In [18]:
m = H_AFR.shape[1]

H_valid = np.zeros((0, m))
for H in [H_AFR, H_AMR, H_EUR, H_EAS]:
    H_valid = np.vstack([H_valid, H[-50:, :]])

results = dict()
for k in ['AFR', 'EUR', 'EAS', 'AMR']:
    results[k] = np.zeros((H_valid.shape[0], 0))

prog = IntProgress(value=0, max=m)
display(prog)

for fname in os.listdir('./saved_models/'):
    f = open('saved_models/%s' % fname, 'rb')
    rbm = pkl.load(f)
    f.close()
    
    if rbm.population != 'EUR':
        continue
    
    results['model_info'].append(str(rbm))
    print(str(rbm))
    
    prog.value = 0
    
    for col in range(m):
        out = rbm.impute(H_valid, position=col).reshape(-1, 1)
        results[rbm.population] = np.hstack([results[rbm.population], out])
        prog.value += 1
        
    fpath = 'data/imputations_t50_TMP.p'
    f = open(fpath, 'wb')
    pkl.dump(results, f)
    f.close()
        
prog.close()

f = open('data/imputations_t50.p', 'wb')
pkl.dump(results, f)
f.close()

IntProgress(value=0, max=16285)

TypeError: file must have a 'write' attribute

# Visualize

In [ ]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def free_energy(rbm, v, eps=1e-5):
    """
    TODO: doc
    """
    a = rbm.intercept_visible_
    v = v.reshape(-1, len(a))
    n = v.shape[0]
    
    term1 = -np.dot(v, a)
    
    b = rbm.intercept_hidden_
    W = rbm.components_
    x = b + np.dot(v, W.T)
    p = sigmoid(x)
    
    p = np.clip(p, eps, 1-eps)
    
    term2 = -(x * p).sum(axis=1)
    
    term3 = np.sum(p * np.log(p) + (1. - p) * np.log(1. - p), axis=1)
    
    return term1 + term2 + term3
    
    
v_AFR = H_AFR[100]
v_EUR = H_EUR[100]
v_EAS = H_EAS[100]
v_AMR = H_AMR[100]

Fv_AFR = free_energy(rbm, H_AFR)
Fv_EUR = free_energy(rbm, H_EUR)
Fv_AMR = free_energy(rbm, H_AMR)
Fv_EAS = free_energy(rbm, H_EAS)

f, ax = plt.subplots(figsize=(12, 7))
# ax.scatter(range(len(Fv_AFR)), Fv_AFR, label='AFR')
# ax.scatter(range(len(Fv_EUR)), Fv_EUR, label='EUR')
# ax.scatter(range(len(Fv_AMR)), Fv_AMR, label='AMR')
# ax.scatter(range(len(Fv_EAS)), Fv_EAS, label='EAS')

ax.scatter(Fv_AFR, [0]*len(Fv_AFR), label='AFR')
ax.scatter(Fv_EUR, [1]*len(Fv_EUR), label='EUR')
ax.scatter(Fv_AMR, [2]*len(Fv_AMR), label='AMR')
ax.scatter(Fv_EAS, [3]*len(Fv_EAS), label='EAS')

plt.legend()
plt.show()
print('AFR: %f' % free_energy(rbm, H_AFR).min())
print('EUR: %f' % free_energy(rbm, H_EUR).min())
print('EAS: %f' % free_energy(rbm, H_EAS).min())
print('AMR: %f' % free_energy(rbm, H_AMR).min())

In [ ]:
mx = 6151
mn = 4060

bins = np.linspace(mn, mx, 50)

hist_AFR, _ = np.histogram(Fv_AFR, bins=bins)
hist_EUR, _ = np.histogram(Fv_EUR, bins=bins)
hist_AMR, _ = np.histogram(Fv_AMR, bins=bins)
hist_EAS, _ = np.histogram(Fv_EAS, bins=bins)

# print(hist_AFR.shape)

# # h_ALL = hist_AFR + hist_EUR + hist_AMR + hist_EAS

# for i in range(len(h_ALL)):
#     s = hist_AFR[i] + hist_AMR[i] + hist_EAS[i] + hist_EUR[i]
#     hist_AFR[i] /= s
#     hist_AMR[i] /= s
#     hist_EAS[i] /= s
#     hist_EUR[i] /= s

f, ax = plt.subplots(figsize=(12, 7))

for i in range(len(bins) - 1):
    bs, be = bins[i], bins[i+1]
    
    bottom = 0
    p_AFR = hist_AFR[i] / h_ALL[i]
    lg = 'AFR' if i == 0 else None
    ax.fill_between(
        [bs, be], [bottom, bottom], [p_AFR, p_AFR], color='blue', label=lg
    )
    
    bottom += p_AFR
    p_AMR = hist_AMR[i] / h_ALL[i]
    lg = 'AMR' if i == 0 else None
    ax.fill_between(
        [bs, be], [bottom, bottom], [bottom+p_AMR, bottom+p_AMR], 
        color='green', label=lg
    )
    
    bottom += p_AMR
    p_EUR = hist_EUR[i] / h_ALL[i]
    lg = 'EUR' if i == 0 else None
    ax.fill_between(
        [bs, be], [bottom, bottom], [bottom+p_EUR, bottom+p_EUR], 
        color='orange', label=lg
    )
    
    bottom += p_EUR
    p_EAS = hist_EUR[i] / h_ALL[i]
    lg = 'EAS' if i == 0 else None
    ax.fill_between(
        [bs, be], [bottom, bottom], [bottom+p_EAS, bottom+p_EAS], 
        color='red', label=lg
    )
    
#     print(i, p_AFR, p_AMR, p_EUR, p_EAS)
    
plt.legend()
plt.show()